In [20]:
import pandas as pd
import numpy as np

In [2]:
weather = pd.read_csv('./datasets/weather.csv')
print(weather.describe())

               date       pressure     wind_speed  wind_direction  \
count  1.026550e+05  102655.000000  102655.000000   102655.000000   
mean   2.015097e+07    5765.919938     130.467292   486575.433023   
std    1.415813e+04   68925.854851   11252.632730   499252.341584   
min    2.013010e+07       0.000000       0.000000        0.000000   
25%    2.014041e+07     969.280000       2.600000      157.000000   
50%    2.015072e+07     987.700000       3.500000      350.000000   
75%    2.016101e+07    1010.300000       4.700000   999007.000000   
max    2.017123e+07  999999.000000  999999.000000   999999.000000   

         temperature       humidity         rain20         rain08  \
count  102655.000000  102655.000000  102655.000000  102655.000000   
mean     2773.339649    2895.843177   38821.923065   41432.580647   
std     52432.213306   53072.003741  193162.610550  199281.055539   
min       -19.000000       0.000000       0.000000       0.000000   
25%         9.600000      62.0000

In [10]:
# print(weather['date'].value_counts())
# print(weather.groupby(weather['date']))
print(weather.head())

       date station city county  pressure  wind_speed  wind_direction  \
0  20130101      巴南  重庆市    巴南市     959.1         3.4          999003   
1  20130101      綦江  重庆市    綦江市     962.6         6.5          999012   
2  20130101      渝北  重庆市    渝北市     963.9         4.1          999003   
3  20130101      梁平  重庆市    梁平市     965.3         2.2          999001   
4  20130101      垫江  重庆市    垫江县     968.9         1.7          999004   

   temperature  humidity  rain20  ...  cloudy  rain  fog haze  dust  thunder  \
0          6.0      48.0     0.0  ...       1     0    0    0     0        0   
1          5.6      59.0     0.0  ...       0     0    0    0     0        0   
2          5.9      44.0     0.0  ...       1     0    0    0     0        0   
3          2.1      68.0     0.0  ...       1     0    1    0     0        0   
4          3.8      61.0     0.0  ...       1     0    0    0     0        0   

   lightning  snow  hail  wind  
0          0     0     0     0  
1          0  

In [188]:
import datetime
from datetime import timedelta

def gen_dates(b_date,days):
    day = timedelta(days=1)
    for i in range(days):
        yield b_date+ day*i

def get_date_list(start_date,end_date):
    start = datetime.datetime.strptime(start_date, "%Y-%m-%d")
    end = datetime.datetime.strptime(end_date,"%Y-%m-%d")
    data = []
    for d in gen_dates(start,((end-start).days+1)):
        data.append(d.strftime("%Y-%-m-%-d"))
    return data


def covert_txt_to_dataframe(txt_path,date_list):
    # 将txt转成dataframe
    with open(txt_path,'r') as f:
        lines = f.read().split('\n')
        df_dict = {
            'date':[],
            'AQI':[],
            'Quality_rank':[],
        }

        for i,start in enumerate(range(8,len(lines)-1,3)):
            try:
                index = len(date_list[i])
                day = lines[start][:index]
                df_dict['date'].append(day)
                df_dict['AQI'].append(lines[start][index:])
                df_dict['Quality_rank'].append(lines[start+1])
            except Exception as e:
                print(lines[start])
                print(e)
    return df_dict

# 北京市
def beijing_convert():
    # 得到时间列表
    date_list = get_date_list('2013-12-2','2017-12-6')
    date_list.pop(date_list.index('2016-4-16'))
    print(len(date_list))
    df_dict = covert_txt_to_dataframe('./datasets/beijing_air.txt',date_list)
    bj_df = pd.DataFrame(df_dict)
    date = pd.to_datetime(df['date'].apply(lambda x: str(x)))
    bj_df['date'] = date
    print(bj_df.info())
    return bj_df
    
# 重庆市
def chongqin_convert():
    # 得到时间列表
    date_list = get_date_list('2013-12-2','2017-12-16')
    print(len(date_list))
    cq_dict = covert_txt_to_dataframe('./datasets/chongqing_air.txt',date_list)
    cq_df = pd.DataFrame(cq_dict)
    date = pd.to_datetime(cq_df['date'].apply(lambda x: str(x)))
    cq_df['date'] = date
    print(cq_df.info())
    return cq_df

# 上海市
def shanghai_convert():
     # 得到时间列表
    date_list = get_date_list('2013-12-2','2017-12-6')
    print(len(date_list))
    sh_dict = covert_txt_to_dataframe('./datasets/shanghai_air.txt',date_list)
    sh_df = pd.DataFrame(sh_dict)
    date = pd.to_datetime(sh_df['date'].apply(lambda x: str(x)))
    sh_df['date'] = date
    print(sh_df.info())
    return sh_df
    

In [192]:
# grouped = weather.groupby(['city','date'])
df_group = weather.groupby([weather['city'],weather['date']]).mean()
df_group.to_csv('./group_weather.csv')
new_weather = pd.read_csv('./group_weather.csv')
new_weather['date'] = pd.to_datetime(new_weather['date'].apply(lambda x: str(x)))

bj_df = beijing_convert()
beijing = new_weather.loc[new_weather['city']=='北京市']
new_beijing = pd.merge(bj_df,beijing,how="inner",on='date') 

cq_df = chongqin_convert()
chongqin = new_weather.loc[new_weather['city']=='重庆市']
new_chongqing = pd.merge(cq_df,chongqin,how='inner',on='date')

sh_df = shanghai_convert()
shanghai = new_weather.loc[new_weather['city']=='上海市']
new_shanghai = pd.merge(sh_df,shanghai,how='inner',on='date')

new_weather1 = pd.concat([new_beijing,new_chongqing,new_shanghai])
new_weather1.to_csv('./datasets/air_weather.csv')

1465
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1465 entries, 0 to 1464
Data columns (total 3 columns):
date            1465 non-null datetime64[ns]
AQI             1465 non-null object
Quality_rank    1465 non-null object
dtypes: datetime64[ns](1), object(2)
memory usage: 34.4+ KB
None
1476
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1476 entries, 0 to 1475
Data columns (total 3 columns):
date            1476 non-null datetime64[ns]
AQI             1476 non-null object
Quality_rank    1476 non-null object
dtypes: datetime64[ns](1), object(2)
memory usage: 34.7+ KB
None
1466
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1466 entries, 0 to 1465
Data columns (total 3 columns):
date            1466 non-null datetime64[ns]
AQI             1466 non-null object
Quality_rank    1466 non-null object
dtypes: datetime64[ns](1), object(2)
memory usage: 34.4+ KB
None
